In [1]:
import os
import glob
import pandas as pd

In [2]:
path_in0 = r"..\Raw" + "\\"
path_out0 = r"..\CSV" + "\\"
file_station = r"..\Station Info.csv"

file_list = glob.glob(path_in0 + "*.dat")
file_list.sort()
# get the file names without the .dat extension
code_list = [os.path.basename(f)[:-4] for f in file_list]

col_list1 = ["Code", "Date", "Discharge"]
col_list2 = ["River", "Station", "Lon", "Lat", "Date", "Discharge"]

In [3]:
df_station = pd.read_csv(file_station)
df_station = df_station[df_station["Station Code"].isin(code_list)]
df_station

No  R-Arctic Net ID Arctic RIMS ID Station Code  \
96    97             5952              6         1802   
142  143             6338             18         3801   
143  144             6341             12         3811   
144  145             6342              3         3821   
148  149             6353                        3871   
185  186             6656              2         9803   
221  222             7142              1        11801   
224  225             7145             14        11808   
253  254             7564              8        70801   
254  255             7565                       70827   
255  256             7566             25        70842   
256  257             7567             23        70844   
257  258             7568             10        70850   
261  262             7629            100        71168   
275  276             7709              5     15565447   

                               Station Name       Continent  \
96                     KOLYMA AT KOLYMSKOYE            Asia   
142                     ANABAR AT SASKULAKH            Asia   
143   Olenek At 7.5 km down of Buur's mouth            Asia   
144                           LENA AT KUSUR            Asia   
148                 YNDIGIRKA AT VORONTSOVO            Asia   
185                       YENISEY AT IGARKA            Asia   
221                          OB AT SALEHARD            Asia   
224                        TAZ AT SIDOROVSK            Asia   
253                SEV.DVINA AT UST' PINEGA          Europe   
254                       PECHORA AT OKSINO          Europe   
255                          ONEGA AT POROG          Europe   
256               MEZEN AT MALONISOGORSKOYE          Europe   
257                   Pechora at Ust-TsilmA          Europe   
261                        Ponoy at Kanevka          Europe   
275             YUKON R AT PILOT STATION AK   North_America   

    Drainage Area, km2 Annual Discharge, km3       Lon      Lat  \
96            526 000               102.6287  158.7200  68.7300   
142            78 800                14.2802  114.0800  71.9700   
143           198 000                36.9844  123.6500  71.8500   
144         2 430 000               528.5711  127.3900  70.6800   
148           305 000                50.1473  147.5300  69.5700   
185         2 440 000               580.1065   86.4800  67.4300   
221         2 950 000               393.9537   66.6000  66.6300   
224           100 000                32.9979   82.2800  66.6000   
253           348 000               105.0598   41.9200  64.1300   
254           312 000               138.1038   52.1800  67.6300   
255            55 700                15.5187   38.4700  63.8200   
256            56 400                20.2363   45.6200  65.0100   
257           248 000               108.4712   52.2800  65.4200   
261            10 200                 3.4289   39.6600  67.1300   
275        831 390.83               203.2589 -162.8829  61.9337   

              River                      Station  
96           Kolyma                    Kolymskoe  
142          Anabar                    Saskylakh  
143          Olenek  7.5 km down of Buur's mouth  
144            Lena                      Kyusyur  
148       Indigirka                   Vorontsovo  
185         Yenisey                       Igarka  
221              Ob                    Salekhard  
224             Taz                    Sidorovsk  
253  Northern Dvina                  Ust' Pinega  
254         Pechora                       Oksino  
255           Onega                        Porog  
256           Mezen             Malonisogorskoye  
257         Pechora                 Ust' Tsil'ma  
261           Ponoy                      Kanevka  
275           Yukon                Pilot Station

In [4]:
for index, row in df_station.iterrows():
    code = row["Station Code"]
    if code == "70850":
        continue

    file_in = path_in0 + f"{code}.dat"
    df = pd.read_csv(file_in, sep="\t", header=None)
    if df.shape[1] == 3:
        pass
    elif df.shape[1] == 4:
        df.drop(3, axis=1, inplace=True)
    else:
        raise ValueError("The number of columns is wrong.")

    df.columns = col_list1
    df["Date"] = pd.to_datetime(df["Date"])
    df.drop("Code", axis=1, inplace=True)

    row = df_station[df_station["Station Code"] == code].iloc[0]
    river = row["River"]
    station = row["Station"]
    df["River"] = river
    df["Station"] = station
    df["Lon"] = row["Lon"]
    df["Lat"] = row["Lat"]
    df = df[col_list2]
    df.sort_values("Date", inplace=True)

    file_out = path_out0 + f"{river}__{station}.csv"
    df.to_csv(file_out, index=False)